In [1]:
import pandas as pd
import re
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\rober\AppData\Local\Temp\ipykernel_18356\3169084638.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
original_file_path = 'C:\\Desktop\\training\\TRAINING\\training.csv'
df = pd.read_csv(original_file_path, delimiter='\t', encoding='utf-8')
corrected_column = 'C:\\Desktop\\training\\TRAINING\\corrected_column.csv'
df.to_csv(corrected_column, index=False, sep=',')
df['Text Transcription'] = df['Text Transcription'].str.lower()
lowercased = 'C:\\Desktop\\training\\TRAINING\\lowercased.csv'
df.to_csv(lowercased, index=False, sep=',')
print(f"Cleaned dataset saved to {lowercased}, this datset has all chars lowered")

df['Text Transcription'] = df['Text Transcription'].replace(r'\b\w+\.(com|org|net|zip|co)\b', '', regex=True)
no_websites = 'C:\\Desktop\\training\\TRAINING\\no_websites.csv'
df.to_csv(no_websites, index=False, sep=',')
print(f"Cleaned dataset saved to {no_websites}, this removed websites")

df['Text Transcription'] = df['Text Transcription'].apply(lambda x: contractions.fix(x))
conc = 'C:\\Desktop\\training\\TRAINING\\conc.csv'
df.to_csv(conc, index=False, sep=',')
print(f"Cleaned dataset saved to {conc}, this removed concs")

Cleaned dataset saved to C:\Desktop\training\TRAINING\lowercased.csv, this datset has all chars lowered
Cleaned dataset saved to C:\Desktop\training\TRAINING\no_websites.csv, this removed websites
Cleaned dataset saved to C:\Desktop\training\TRAINING\conc.csv, this removed concs


In [3]:
#removing special chars and handling multiple spaces

def remove_special_characters(text):
    # Remove single quotes without adding a space
    cleaned_text = re.sub(r"'", '', text)
    
    # Replace other special characters with spaces
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', cleaned_text)
    
    # Replace two or more consecutive whitespace characters with a single space
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)
    
    return cleaned_text.strip()  # Remove leading and trailing spaces

df['Text Transcription'] = df['Text Transcription'].apply(remove_special_characters)
no_multiple_space_or_special_char = 'C:\\Desktop\\training\\TRAINING\\no_multiple_space_or_special_char.csv'
df.to_csv(no_multiple_space_or_special_char, index=False)
print(f"Cleaned dataset saved to {no_multiple_space_or_special_char}, removing special chars and handling multiple spaces")

Cleaned dataset saved to C:\Desktop\training\TRAINING\no_multiple_space_or_special_char.csv, removing special chars and handling multiple spaces


In [4]:
#optional (no nums)
def remove_numbers(text):
    # Remove numbers
    cleaned_text = re.sub(r'\d', '', text)
    return cleaned_text.strip()  # Remove leading and trailing spaces

df['Text Transcription'] = df['Text Transcription'].apply(remove_numbers)
no_num = 'C:\\Desktop\\training\\TRAINING\\(final)no_num.csv'
df.to_csv(no_num, index=False)
print(f"Cleaned dataset saved to {no_num}, removing numbers")


Cleaned dataset saved to C:\Desktop\training\TRAINING\(final)no_num.csv, removing numbers


In [5]:

# Tokenize the text into words
df['Text Transcription'] = df['Text Transcription'].apply(word_tokenize)

# Remove stopwords
stop_words = set(stopwords.words('english'))
df['Text Transcription'] = df['Text Transcription'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

# Join the tokenized words back into sentences for TfidfVectorizer
df['Text Transcription'] = df['Text Transcription'].apply(lambda x: ' '.join(x))

# Calculate TF-IDF scores
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Text Transcription'])

# Get feature names (words)
feature_names = tfidf_vectorizer.get_feature_names_out()

# Create a DataFrame with TF-IDF scores
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# Sum the TF-IDF scores across all documents (rows) for each word
word_tfidf_sums = tfidf_df.sum()

# Print the words and their TF-IDF scores
print("Words and their TF-IDF scores:")
print(word_tfidf_sums.sort_values(ascending=False))

Words and their TF-IDF scores:
women         187.961589
like          144.112443
woman         120.236758
girlfriend    106.869148
get            98.219847
                 ...    
clones          0.036755
homed           0.036755
dow             0.034616
horned          0.034616
wolverine       0.034616
Length: 18006, dtype: float64


In [6]:
# Print the words with the highest TF-IDF scores
print("Words with the highest TF-IDF scores:")
print(word_tfidf_sums.nlargest(20))

# Print the words with the lowest TF-IDF scores
print("Words with the lowest TF-IDF scores:")
print(word_tfidf_sums.nsmallest(20))




Words with the highest TF-IDF scores:
women         187.961589
like          144.112443
woman         120.236758
girlfriend    106.869148
get            98.219847
wife           96.974086
kitchen        93.036649
men            87.477241
girl           85.636967
girls          83.413993
one            81.037913
man            80.804864
house          80.404756
call           76.069248
make           75.962371
want           74.774163
know           68.309412
female         65.147150
see            61.380508
people         61.078490
dtype: float64
Words with the lowest TF-IDF scores:
dow               0.034616
horned            0.034616
wolverine         0.034616
clones            0.036755
homed             0.036755
istocrastina      0.052496
sabers            0.052496
amer              0.054205
asla              0.054205
bandhan           0.054205
frou              0.054205
garad             0.054205
joen              0.054205
kamla             0.054205
pasin             0.054205
petma

In [7]:
import pandas as pd


# Create a DataFrame for the words with the highest TF-IDF scores
highest_tfidf_words = word_tfidf_sums.nlargest(20).reset_index()
highest_tfidf_words.columns = ['Word', 'TF-IDF Score']

# Create a DataFrame for the words with the lowest TF-IDF scores
lowest_tfidf_words = word_tfidf_sums.nsmallest(20).reset_index()
lowest_tfidf_words.columns = ['Word', 'TF-IDF Score']

# Merge both DataFrames
tfidf_words_combined = pd.concat([highest_tfidf_words, lowest_tfidf_words], axis=0)

# Save the combined DataFrame to a CSV file
tfidf_words_combined.to_csv('tfidf_scores_combined.csv', index=False)
